In [ ]:
import trimesh
import os
import numpy as np

def load_mesh(file_path):
    """
    Load a .glb file and return a Trimesh Scene or Mesh object.

    Args:
        file_path (str): The path to the .glb file.

    Returns:
        trimesh.Scene or trimesh.Trimesh: The loaded 3D object.
    """
    scene_or_mesh = trimesh.load(file_path)
    return scene_or_mesh

def save_screenshot(mesh, output_folder, angles, distances):
    """
    Render a mesh from multiple angles and distances, and save screenshots.

    Args:
        mesh (trimesh.Trimesh): The mesh to render.
        output_folder (str): The folder to save the screenshots.
        angles (list of tuples): List of angles (yaw, pitch, roll) to capture.
        distances (list of float): List of distances from which to capture images.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Create a scene for the mesh
    scene = trimesh.Scene(mesh)
    
    # Capture images from different angles and distances
    for distance in distances:
        for angle in angles:
            print(f"Using distance: {distance} and angles: {angle}")
            # Set the camera distance
            scene.camera_transform = trimesh.transformations.translation_matrix([0, 0, distance])
            # Set the camera angles
            scene.set_camera(angles=angle)
            # Render the scene and save the image
            png = scene.save_image(resolution=(1920, 1080), visible=True)
            angle_int = tuple(map(int, angle))  # Convert angles to integers
            angle_str = '_'.join(map(str, angle_int))
            distance_str = str(distance)
            output_path = os.path.join(output_folder, f"{angle_str}_{distance_str}.png")
            print(f"Saving screenshot to {output_path}")
            with open(output_path, 'wb') as f:
                f.write(png)

# Example usage
file_paths = [
    (r'C:\Users\Yousef.AlHammadi\Desktop\trojan-cv\assets\3D\Set of PVC pipe accessories kitbash\Each Alone As glb\Connect90Degree.glb', 'Connect90Degree'),
    (r'C:\Users\Yousef.AlHammadi\Desktop\trojan-cv\assets\3D\Set of PVC pipe accessories kitbash\Each Alone As glb\Connect45Degree.glb', 'Connect45Degree')
]

# Generate angles in radians (yaw, pitch, roll) using for loops
angles = []
angle_steps = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4]
for yaw in angle_steps:
    for pitch in angle_steps:
        for roll in angle_steps:
            angles.append((yaw, pitch, roll))

# Distances from the object
distances = [4, 6]

# Base directory for the output
output_base_dir = './output'

for file_path, label in file_paths:
    # Load the mesh or scene from the file
    scene_or_mesh = load_mesh(file_path)
    
    # Create an output folder for each file
    output_folder = os.path.join(output_base_dir, label)
    
    # Check if the loaded object is a Scene or Mesh
    print(f"Checking if the loaded object from {file_path} is a Scene or Mesh...")
    if isinstance(scene_or_mesh, trimesh.Scene):
        print("The loaded object is a Scene. Combining all geometries in the scene into one mesh...")
        combined = trimesh.util.concatenate([mesh for mesh in scene_or_mesh.geometry.values()])
        print("Saving screenshots of the combined mesh...")
        save_screenshot(combined, output_folder, angles, distances)
    elif isinstance(scene_or_mesh, trimesh.Trimesh):
        print("The loaded object is a Mesh. Saving screenshots of the mesh...")
        save_screenshot(scene_or_mesh, output_folder, angles, distances)
    else:
        print("Loaded object is neither a Scene nor a Mesh")
